In [ ]:
# Installation de la bibliothèque PyTest pour les tests
!pip install pytest
# Installation de la bibliothèque Hypothesis pour les tests de propriétés
!pip install hypothesis

## Classe Book

Cette section contient la définition de la classe `Book` et les tests associés pour vérifier la création correcte des instances de livres et la gestion des erreurs.

In [ ]:
# Import de pytest pour les tests
import pytest

# Définition de la classe Book
class Book:
    def __init__(self, title, author):
        if not title or not author:
            raise ValueError("Title and author cannot be empty")
        self.title = title
        self.author = author

    def __repr__(self):
        return f"Book(title='{self.title}', author='{self.author}')"

# Tests pour la classe Book

def test_create_book_with_valid_data():
    # Test la création d'un livre avec des données valides
    book = Book("Harry Potter", "JK Rowling")
    assert book.title == "Harry Potter"
    assert book.author == "JK Rowling"

def test_create_book_with_empty_title_raises_error():
    # Test la levée d'une erreur si le titre est vide
    with pytest.raises(ValueError):
        Book("", "JK Rowling")

def test_create_book_with_empty_author_raises_error():
    # Test la levée d'une erreur si l'auteur est vide
    with pytest.raises(ValueError):
        Book("Harry Potter", "")

## Classe BookManager

Cette section contient la définition de la classe `BookManager` et les tests associés pour vérifier l'ajout de livres à la collection et la récupération de la liste des livres dans l'ordre alphabétique.

In [ ]:
# Classe pour gérer la liste des livres
class BookManager:
    def __init__(self):
        self.books = []

    def add_book(self, book):
        # Vérifie si l'objet ajouté est une instance de Book
        if not isinstance(book, Book):
            raise TypeError("Only Book instances can be added")
        self.books.append(book)

    def list_books(self):
        # Retourne la liste des livres triée par titre
        return sorted(self.books, key=lambda book: book.title)

# Tests pour BookManager

def test_add_book():
    # Test l'ajout d'un livre au gestionnaire
    manager = BookManager()
    book = Book("Harry Potter", "JK Rowling")
    manager.add_book(book)
    assert book in manager.books

def test_list_books_in_alphabetical_order():
    # Test si les livres sont listés dans l'ordre alphabétique
    manager = BookManager()
    book1 = Book("Animal Farm", "George Orwell")
    book2 = Book("1984", "George Orwell")
    manager.add_book(book1)
    manager.add_book(book2)
    assert manager.list_books() == [book2, book1]

def test_list_books_contains_all_books():
    # Test si la liste des livres contient tous les livres ajoutés
    manager = BookManager()
    book1 = Book("Animal Farm", "George Orwell")
    book2 = Book("1984", "George Orwell")
    book3 = Book("Harry Potter", "JK Rowling")
    manager.add_book(book1)
    manager.add_book(book2)
    manager.add_book(book3)
    assert set(manager.list_books()) == {book1, book2, book3}

Actions possibles :

Créer un livre : La classe `BookManager` permet de créer un livre et de l'ajouter à une liste.

Lister tous les livres par ordre alphabétique : La méthode `list_books` dans BookManager trie et retourne les livres par ordre alphabétique du titre.

Ajout d'un livre : La méthode `add_book` dans BookManager permet d'ajouter un livre.

Règles sur les attributs du livre : Des vérifications sont effectuées pour s'assurer que le titre et l'auteur du livre ne sont pas vides.

Récupération et tri des livres : La méthode `list_books` retourne la liste des livres triée par ordre alphabétique du titre.

## Classe Interface BookDatabasePort

Cette section contient la définition de la classe `BookDatabasePort`. L'interface `BookDatabasePort` et son implémentation `BookDatabaseImplementation` servent de ports pour une future intégration avec une base de données.

In [ ]:
from abc import ABC, abstractmethod

# Interface abstraite pour la base de données des livres
class BookDatabasePort(ABC):
    @abstractmethod
    def add_book(self, book):
        pass

    @abstractmethod
    def get_all_books(self):
        pass

# Implémentation concrète de l'interface pour la base de données
class BookDatabaseImplementation(BookDatabasePort):
    def __init__(self):
        # Initialisation de la connexion à la base de données
        pass

    def add_book(self, book):
        # Logique pour ajouter un livre à la base de données
        pass

    def get_all_books(self):
        # Logique pour récupérer tous les livres de la base de données
        pass

# Modification de la classe BookManager pour utiliser le port de base de données
class BookManager:
    def __init__(self, db_port: BookDatabasePort):
        self.db_port = db_port
        self.books = []

    def add_book(self, book):
        self.db_port.add_book(book)
        self.books.append(book)

    def list_books(self):
        return sorted(self.books, key=lambda book: book.title)

## Test de Propriétés

Cette section contient les tests de propriétés.

In [ ]:
# Importation de la bibliothèque Hypothesis
from hypothesis import given, strategies as st

# Test de propriété pour vérifier que la liste des livres retournés contient tous les éléments de la liste stockée
def test_book_list_contains_all_books(book_manager):
    @given(st.lists(st.builds(Book, title=st.text(), author=st.text())))
    def property_test(books):
        for book in books:
            book_manager.add_book(book)
        assert set(book_manager.list_books()) == set(books)
    property_test()